In [1]:
import pandas as pd
import os
import datetime
import re

## 1. Keep Only News from Newspapers

In [ ]:
# All parsed XML files
df=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/parsed_xml.pkl')
print(df.info())

In [4]:
print(df['Type'].value_counts())

News                      848427
Feature                     4060
Undefined                    668
General Information           32
Commentary                    31
Article                       30
Statistics/Data Report        10
Obituary                       8
Review                         7
Correction/Retraction          5
Recipe                         3
Interview                      2
Editorial                      1
Name: Type, dtype: int64


In [5]:
df=df[df['Type']=='News'].reset_index(drop=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 848427 entries, 0 to 848426
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   ID             848427 non-null  object
 1   Title          848427 non-null  object
 2   Type           848427 non-null  object
 3   StartDate      848427 non-null  object
 4   EndDate        848427 non-null  object
 5   Text           848427 non-null  object
 6   TextWordCount  848427 non-null  object
 7   PubTitle       848427 non-null  object
 8   SourceType     848427 non-null  object
dtypes: object(9)
memory usage: 58.3+ MB
None


In [6]:
print(df['SourceType'].value_counts())

Newspapers    848427
Name: SourceType, dtype: int64


## 2. Remove Overlaps between Sources

In [7]:
print(df['PubTitle'].value_counts())

Wall Street Journal                        152069
New York Times                             125270
Wall Street Journal (Online)                99914
The Washington Post                         93086
Los Angeles Times                           68818
Los Angeles Times (pre-1997 Fulltext)       59421
Chicago Tribune                             55013
Boston Globe                                54071
Chicago Tribune (pre-1997 Fulltext)         38858
The Washington Post (pre-1997 Fulltext)     31037
Boston Globe (pre-1997 Fulltext)            23916
USA TODAY                                   21021
USA TODAY (pre-1997 Fulltext)               11773
USA Today (Online)                           7458
Boston Globe (Online)                        6702
Name: PubTitle, dtype: int64


In [8]:
# Convert dates
df['StartDate']=df['StartDate'].astype('datetime64[ns]')
df['Year']=df['StartDate'].astype('datetime64[ns]').dt.year
df['Month']=df['StartDate'].astype('datetime64[ns]').dt.month

In [9]:
pubtitles=['USA TODAY (pre-1997 Fulltext)','USA TODAY','USA Today (Online)']
for title in pubtitles:
    print(df[df['PubTitle']==title].sort_values('StartDate')[['StartDate']])
print(pubtitles)

        StartDate
833650 1987-04-01
288936 1987-04-01
825813 1987-04-01
645156 1987-04-01
135256 1987-04-02
...           ...
188669 1997-02-12
703759 1997-02-12
824499 1997-02-12
819079 1997-02-14
241031 1997-02-14

[11773 rows x 1 columns]
        StartDate
533213 1989-01-11
470377 1989-01-13
1414   1989-01-13
521988 1989-01-19
506743 1989-01-23
...           ...
574687 2020-08-31
404341 2020-08-31
499195 2020-08-31
719228 2020-08-31
662371 2020-08-31

[21021 rows x 1 columns]
        StartDate
269473 2016-05-10
54772  2016-05-12
654157 2016-05-13
426357 2016-05-13
762926 2016-05-14
...           ...
574408 2020-08-31
675044 2020-08-31
165394 2020-08-31
731028 2020-08-31
707856 2020-08-31

[7458 rows x 1 columns]
['USA TODAY (pre-1997 Fulltext)', 'USA TODAY', 'USA Today (Online)']


In [10]:
# Clean duplicated news articles due to overlapped databases
df=df[((df['PubTitle']=='Boston Globe') & (df['StartDate']>datetime.datetime(1996,12,31)))
                  | (df['PubTitle']!='Boston Globe')]
df=df[((df['PubTitle']=='Chicago Tribune') & (df['StartDate']>datetime.datetime(1996,12,3)))
                  | (df['PubTitle']!='Chicago Tribune')]
df=df[((df['PubTitle']=='Los Angeles Times') & (df['StartDate']>datetime.datetime(1996,12,3)))
                  | (df['PubTitle']!='Los Angeles Times')]
df=df[((df['PubTitle']=='USA TODAY') & (df['StartDate']>datetime.datetime(1997,2,14)))
                  | (df['PubTitle']!='USA TODAY')]
df=df[((df['PubTitle']=='The Washington Post') & (df['StartDate']>datetime.datetime(1996,12,3)))
                  | (df['PubTitle']!='The Washington Post')]

In [11]:
# Consolidate newspaper names
df.loc[(df['PubTitle']=='Boston Globe (pre-1997 Fulltext)') | (df['PubTitle']=='Boston Globe') | 
       (df['PubTitle']=='Boston Globe (Online)'),'Newspaper']='Boston Globe'
df.loc[(df['PubTitle']=='Wall Street Journal') | (df['PubTitle']=='Wall Street Journal (Online)'),
    'Newspaper']='Wall Street Journal'
df.loc[(df['PubTitle']=='USA TODAY (pre-1997 Fulltext)') | (df['PubTitle']=='USA TODAY') | 
       (df['PubTitle']=='USA Today (Online)'),'Newspaper']='USA Today'
df.loc[(df['PubTitle']=='Chicago Tribune (pre-1997 Fulltext)') | (df['PubTitle']=='Chicago Tribune') | 
       (df['PubTitle']=='Chicago Tribune (Online)'),'Newspaper']='Chicago Tribune'
df.loc[(df['PubTitle']=='Los Angeles Times') | (df['PubTitle']=='Los Angeles Times (pre-1997 Fulltext)') | 
        (df['PubTitle']=='Los Angeles Times (Online)'),'Newspaper']='Los Angeles Times'
df.loc[(df['PubTitle']=='New York Times') | (df['PubTitle']=='New York Times (Online)'),'Newspaper']='New York Times'
df.loc[(df['PubTitle']=='The Washington Post') | (df['PubTitle']=='The Washington Post (pre-1997 Fulltext)') | 
       (df['PubTitle']=='The Washington Post (Online)'),'Newspaper']='The Washington Post'

In [12]:
df=df.sort_values(['Newspaper','StartDate','Title']).reset_index(drop=True)

In [13]:
print(df.info())
print(df['Newspaper'].value_counts(dropna=False))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 822737 entries, 0 to 822736
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   ID             822737 non-null  object        
 1   Title          822737 non-null  object        
 2   Type           822737 non-null  object        
 3   StartDate      822737 non-null  datetime64[ns]
 4   EndDate        822737 non-null  object        
 5   Text           822737 non-null  object        
 6   TextWordCount  822737 non-null  object        
 7   PubTitle       822737 non-null  object        
 8   SourceType     822737 non-null  object        
 9   Year           822737 non-null  int64         
 10  Month          822737 non-null  int64         
 11  Newspaper      822737 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 75.3+ MB
None
Wall Street Journal    251983
New York Times         125270
Los Angeles Times      121

In [14]:
print(df['PubTitle'].value_counts())
print(df['PubTitle'].nunique())

Wall Street Journal                        152069
New York Times                             125270
Wall Street Journal (Online)                99914
The Washington Post                         85735
Los Angeles Times                           61985
Los Angeles Times (pre-1997 Fulltext)       59421
Chicago Tribune                             51165
Boston Globe                                48304
Chicago Tribune (pre-1997 Fulltext)         38858
The Washington Post (pre-1997 Fulltext)     31037
Boston Globe (pre-1997 Fulltext)            23916
USA TODAY                                   19130
USA TODAY (pre-1997 Fulltext)               11773
USA Today (Online)                           7458
Boston Globe (Online)                        6702
Name: PubTitle, dtype: int64
15


In [15]:
print(df.head())

          ID                                            Title  Type  \
0  294326637             1984: IT WAS THE YEAR OF THE BIG LIE  News   
1  294308147  CONTROLS LIFTED ON ABOUT HALF OF US NATURAL GAS  News   
2  294323196              COURT UPHOLDS DIABLO CANYON LICENSE  News   
3  294311708                      HUNT'S IMMACULATE RECEPTION  News   
4  294262284                          LEGISLATIVE REPORT CARD  News   

   StartDate     EndDate                                               Text  \
0 1985-01-01  1985-01-01  ART BUCHWALD Art Buchwald is a syndicated colu...   
1 1985-01-01  1985-01-01  After 30 years of strict federal control, pric...   
2 1985-01-01  1985-01-01  A federal appeals court yesterday upheld the N...   
3 1985-01-01  1985-01-01  COTTON BOWL '85 / JOHN ROBINSON John Robinson ...   
4 1985-01-01  1985-01-01  Much of Beacon Hill's 1984 legislative activit...   

  TextWordCount                          PubTitle  SourceType  Year  Month  \
0           422  Bos

In [16]:
df.to_pickle('/home/ec2-user/SageMaker/New Uncertainty/parsed_xml.pkl')

## 3. Identify Duplicated Articles

In [48]:
df=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/parsed_xml.pkl')
print("Number of articles:",len(df))

Number of articles: 822737


In [55]:
# Full text for certain articles is not available due to copyright restrictions
print("Number of empty full texts:",df[df['Text']==""]['ID'].nunique())
print(df[df['Text']==""]['Newspaper'].value_counts())
# # Examples
# print(df[df['Text']==""]['ID'][-10:])
# print(df[df['Text']==""]['Title'][-10:])

Number of empty full texts: 18544
New York Times         7073
Wall Street Journal    5382
Boston Globe           5072
USA Today               910
Los Angeles Times        42
Chicago Tribune          34
The Washington Post      31
Name: Newspaper, dtype: int64


In [ ]:
# Remove spaces and lower text
text_lower=[]
for text in df['Text']:
    new=re.sub(' +',' ',text.replace('\n',' ').lower()).strip()
    text_lower.append(new)
print(len(text_lower))

In [ ]:
df['TextLower']=text_lower

In [ ]:
# Remove unavailable full text and sort
df=df[df['TextLower']!=""].sort_values(['TextLower','StartDate']).reset_index(drop=True)
print(len(df))

In [41]:
# Check duplicates
df['GroupNo']=df.groupby('TextLower').cumcount()+1
print("Number of duplicated articles:",df[df['GroupNo']>1]['ID'].nunique())

15677


In [ ]:
print(df[['ID','Title','TextLower','GroupNo']][181:196])

In [38]:
# Keep the earliest article
df_nodup=df.groupby('TextLower').nth(0).reset_index()
print(df_nodup.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788516 entries, 0 to 788515
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   TextLower      788516 non-null  object        
 1   ID             788516 non-null  object        
 2   Title          788516 non-null  object        
 3   Type           788516 non-null  object        
 4   StartDate      788516 non-null  datetime64[ns]
 5   EndDate        788516 non-null  object        
 6   Text           788516 non-null  object        
 7   TextWordCount  788516 non-null  object        
 8   PubTitle       788516 non-null  object        
 9   SourceType     788516 non-null  object        
 10  Year           788516 non-null  int64         
 11  Month          788516 non-null  int64         
 12  Newspaper      788516 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(10)
memory usage: 78.2+ MB
None


In [43]:
df_nodup['GroupNo']=df_nodup.groupby('TextLower').cumcount()+1
print("Number of duplicated articles:", df_nodup[df_nodup['GroupNo']>1]['ID'].nunique())
print("Number of unavailable articles:",df_nodup[df_nodup['TextLower']==""]['ID'].nunique())

Number of duplicated articles: 0
Number of unavailable articles: 0


In [45]:
df_nodup[['ID']].to_csv('/home/ec2-user/SageMaker/New Uncertainty/IDs_no_duplicates.csv',index=False)

## A1. Newspaper Articles

In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 822737 entries, 0 to 822736
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   ID             822737 non-null  object        
 1   Title          822737 non-null  object        
 2   Type           822737 non-null  object        
 3   StartDate      822737 non-null  datetime64[ns]
 4   EndDate        822737 non-null  object        
 5   Text           822737 non-null  object        
 6   TextWordCount  822737 non-null  object        
 7   PubTitle       822737 non-null  object        
 8   SourceType     822737 non-null  object        
 9   Year           822737 non-null  int64         
 10  Month          822737 non-null  int64         
 11  Newspaper      822737 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 75.3+ MB
None


In [11]:
print(df['Newspaper'].value_counts())

Wall Street Journal    251983
New York Times         125270
Los Angeles Times      121406
The Washington Post    116772
Chicago Tribune         90023
Boston Globe            78922
USA Today               38361
Name: Newspaper, dtype: int64


## A2. An Example:

In [10]:
print("TITLE:", df['Title'][0])
print("NEWSPAPER:",df['Newspaper'][0])
print("PUBLICATION DATE:",df['StartDate'][0])
print("FULL TEXT:")
print(re.sub(' +',' ',df['Text'][0].replace('\n',' ')).strip())

TITLE: 1984: IT WAS THE YEAR OF THE BIG LIE
NEWSPAPER: Boston Globe
PUBLICATION DATE: 1985-01-01 00:00:00
FULL TEXT:
ART BUCHWALD Art Buchwald is a syndicated columnist. A lot of people lied to all of us in 1984. Here are just a few of the whoppers I remember. "I'm from the telephone company, and I'm here to save you money." "The rise in the price of a postage stamp from 20 to 22 cents will speed up the delivery of a first-class letter." "Every dollar in the Defense Department budget will be accounted for." "With the new human services cuts I have recommended, no one truly in need will suffer." "By merging these two great companies we will cut costs, increase productivity and provide better services without laying off one single person." "If I am elected, I will not be beholden to any special interest group." "We're willing to negotiate with the Americans seriously on arms control." "We're willing to negotiate seriously with the Soviets on arms control." "The CIA has no intention of ov